In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# %load ../../loaders/imports.py
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import time
import pdb

# Add the uoicorr directory to the path
sys.path.append('../../../uoicorr_run')

# Add the root directory of this repository
sys.path.append('../..')

from postprocess_utils import *
import pandas as pd
import sqlalchemy

In [5]:
from utils import gen_data, gen_covariance, sparsify_beta, gen_beta2

In [6]:
root_dir = '/media/akumar/Data/nse/'

In [7]:
# Read the non-concatenated dataframes to ensure indices are properly preserved
lasso = pd.read_pickle('%s/finalfinal/lasso_df.dat' % root_dir)
mcp = pd.read_pickle('%s/finalfinal/mcp_df.dat' % root_dir)
scad = pd.read_pickle('%s/finalfinal/scad_df.dat' % root_dir)
en = pd.read_pickle('%s/finalfinal/en_df.dat' % root_dir)

In [8]:
import h5py

In [9]:
# Iterate through each dataframe and isolate the sets of reps and calculate the bias and variance of estimates
# 1) along the entire vector, 2) along the properly selected coefficients

In [10]:
import itertools

In [33]:
460772 * 4/50/3600

10.239377777777777

In [29]:
dframes = [lasso, mcp, scad, en]
sparsity = np.unique(lasso['sparsity'].values)
betawidth = np.unique(lasso['betawidth'].values)
selection_methods = np.unique(lasso['selection_method'].values)
kappa = np.unique(lasso['kappa'].values)
np_ratio = np.unique(lasso['np_ratio'].values)
cov_idxs = np.arange(80)

beta_fnames = ['%s/finalfinal/%s_pp_beta.h5' % (root_dir, dfname) for dfname in ['lasso', 'mcp', 'scad', 'en']]
beta_files = [h5py.File(beta_fname, 'r') for beta_fname in beta_fnames]

param_combos = itertools.product(sparsity, betawidth, selection_methods, cov_idxs, kappa, np_ratio)

for i, dframe in enumerate(dframes):
    for param_combo in param_combos:
        s, bw, sm, cidx, kappa, npr = param_combo
        df = apply_df_filters(dframe, sparsity=s, betawidth=bw, 
                                  selection_method=sm, cov_idx=cidx, kappa=kappa, np_ratio=npr)
        if df.shape[0] == 0:
            continue
        else:
            print('Beginning task!')
            try:
                assert(df.shape[0] == 20)
            except:
                pdb.set_trace()
            bias1, bias2, var = calc_bias_var(df, beta_files[i])

Beginning task!
0.020168542861938477
Beginning task!
0.02080845832824707
Beginning task!
0.020191192626953125
Beginning task!
0.020018815994262695
Beginning task!
0.02045750617980957
Beginning task!
0.02052617073059082
Beginning task!
0.020495176315307617
Beginning task!
0.02040386199951172
Beginning task!
0.020600080490112305
Beginning task!
0.020561695098876953
Beginning task!
0.020243406295776367
Beginning task!
0.020472288131713867
Beginning task!
0.02054452896118164
Beginning task!
0.020373821258544922
Beginning task!
0.02042698860168457
Beginning task!
0.020612239837646484
Beginning task!
0.021223068237304688
Beginning task!
0.020627975463867188
Beginning task!
0.0208890438079834
Beginning task!
0.02079606056213379
Beginning task!
0.021254777908325195
Beginning task!
0.020152807235717773
Beginning task!
0.020513534545898438
Beginning task!
0.02063131332397461
Beginning task!
0.02083420753479004
Beginning task!
0.02037668228149414
Beginning task!
0.020412683486938477


KeyboardInterrupt: 

In [28]:
def calc_bias_var(df, beta_file):
    
    indices = list(df.index)
    
    # Take the indices
    beta = beta_file['beta'][indices, :]
    # Ensure all the betas are the same
    assert(np.isclose(beta, beta[0]).all())
    
    beta_hats = beta_file['beta_hat'][indices, :]
    
    # Total bias
    total_bias = np.mean(np.linalg.norm(beta - beta_hats, axis=1))
    
    common_support_bias = 0
    
    # Common support bias
    for i in range(len(indices)):
        common_support = list(set(np.nonzero(beta[i, :])[0]).intersection(set(np.nonzero(beta_hats[i, :])[0])))
        common_support_bias += 1/len(indices) * np.linalg.norm(beta[i, common_support] - beta_hats[i, common_support])
    variance = np.mean(np.var(beta_hats, axis = 0))

    return total_bias, common_support_bias, variance